In [1]:
import random
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import torch,torchvision
from torch.nn import *
from tqdm import tqdm
import cv2
from torch.optim import *
# Preproccessing
from sklearn.preprocessing import (
    StandardScaler,
    RobustScaler,
    MinMaxScaler,
    MaxAbsScaler,
    OneHotEncoder,
    Normalizer,
    Binarizer
)
# Decomposition
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
# Feature Selection
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import SelectFromModel
# Model Eval
from sklearn.compose import make_column_transformer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error
# Other
import pickle
import wandb

PROJECT_NAME = 'Learn-Time-Series-Forecasting-From-Gold-Price'
device = 'cuda:0'
np.random.seed(21)
random.seed(21)
torch.manual_seed(21)

In [4]:
data = torch.from_numpy(np.array(pd.read_csv('./data.csv')['Value'].tolist())).view(1,-1).to(device).float()

In [7]:
len(data[0])

10787

In [8]:
data_input = data[:1,:-1].to(device).float()
data_target = data[:1,1:].to(device).float()

In [9]:
class Model(Module):
    def __init__(self):
        super().__init__()
        self.hidden = 512
        self.lstm1 = LSTMCell(1,self.hidden)
        self.lstm2 = LSTMCell(self.hidden,self.hidden)
        self.linear = Linear(self.hidden,1)
    
    def forward(self,X,future=0):
        preds = []
        batch_size = X.shape[0]
        h_t1 = torch.zeros(batch_size,self.hidden).to(device)
        c_t1 = torch.zeros(batch_size,self.hidden).to(device)
        h_t2 = torch.zeros(batch_size,self.hidden).to(device)
        c_t2 = torch.zeros(batch_size,self.hidden).to(device)
        for X_batch in X.split(1,dim=1):
            X_batch = X_batch.to(device)
            h_t1,c_t1 = self.lstm1(X_batch,(h_t1,c_t1))
            h_t1 = h_t1.to(device)
            c_t1 = c_t1.to(device)
            h_t2,c_t2 = self.lstm2(h_t1,(h_t2,c_t2))
            h_t2 = h_t2.to(device)
            c_t2 = c_t2.to(device)
            pred = self.linear(h_t2)
            preds.append(pred)
        for _ in range(future):
            h_t1,c_t1 = self.lstm1(X_batch,(h_t1,c_t1))
            h_t1 = h_t1.to(device)
            c_t1 = c_t1.to(device)
            h_t2,c_t2 = self.lstm2(h_t1,(h_t2,c_t2))
            h_t2 = h_t2.to(device)
            c_t2 = c_t2.to(device)
            pred = self.linear(h_t2)
            preds.append(pred)
        preds = torch.cat(preds,dim=1)
        return preds

In [10]:
model = Model().to(device)

In [14]:
optimizer = LBFGS(model.parameters(),lr=0.001)

In [15]:
criterion = MSELoss()

In [16]:
epochs = 100

In [17]:
epochs_iter = tqdm(range(epochs))
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in epochs_iter:
    def closure():
        epochs_iter.set_description('optimizer.zero_grad()')
        optimizer.zero_grad()
        epochs_iter.set_description('preds = model(data_input)')
        preds = model(data_input)
        epochs_iter.set_description('loss = criterion(preds,data_target)')
        loss = criterion(preds,data_target)
        epochs_iter.set_description('loss.backward()')
        loss.backward()
        epochs_iter.set_description("wandb.log({'Loss':loss.item()})")
        wandb.log({'Loss':loss.item()})
        return loss
    epochs_iter.set_description('optimizer.step(closure)')
    optimizer.step(closure)
    epochs_iter.set_description('with torch.no_grad():')
    with torch.no_grad():
        epochs_iter.set_description('future = 100')
        future = 100
        epochs_iter.set_description('preds = model(data_input,future)')
        preds = model(data_input,future)
        epochs_iter.set_description('loss = criterion(preds[:,:-future],data_target)')
        loss = criterion(preds[:,:-future],data_target)
        epochs_iter.set_description("andb.log({'Val Loss':loss.item()})")
        wandb.log({'Val Loss':loss.item()})
        epochs_iter.set_description('preds = preds[0].cpu().view(-1).detach().numpy()')
        preds = preds[0].cpu().view(-1).detach().numpy()
        epochs_iter.set_description('n = dataa_input.shape[1]')
        n = data_input.shape[1]
        epochs_iter.set_description('plt.figure(figsize=(12,7))')
        plt.figure(figsize=(12,7))
        plt.plot(np.arange(n),data_target.cpu().view(-1).detach().numpy())
        plt.plot(np.arange(n,n+future),preds[n:])
        plt.savefig('./img.png')
        plt.close()
        wandb.log({'Img':wandb.Image(cv2.imread('./img.png'))})
wandb.finish()

  0%|                                                   | 0/100 [00:00<?, ?it/s]wandb: Currently logged in as: ranuga-d (use `wandb login --relogin` to force relogin)


loss.backward():   2%|▍                      | 2/100 [05:01<3:23:48, 124.78s/it]Process wandb_internal:
Traceback (most recent call last):
  File "/home/indika/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/indika/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/indika/anaconda3/lib/python3.7/site-packages/wandb/sdk/internal/internal.py", line 152, in wandb_internal
    thread.join()
  File "/home/indika/anaconda3/lib/python3.7/threading.py", line 1044, in join
    self._wait_for_tstate_lock()
  File "/home/indika/anaconda3/lib/python3.7/threading.py", line 1060, in _wait_for_tstate_lock
    elif lock.acquire(block, timeout):
KeyboardInterrupt
loss.backward():   2%|▍                      | 2/100 [05:04<4:08:42, 152.27s/it]


KeyboardInterrupt: 